In [2]:
import numpy as np
from numpy.linalg import solve, norm



In [3]:
def primal_dual_ipm(c, A, b, max_iter=10000, tol=1e-8, mu=0.1):
    """
    Solve Linear Programming problem using Primal-Dual Path-Following Interior Point Method
    
    minimize    c^T x
    subject to  Ax = b
                x >= 0
    
    Parameters:
    -----------
    c : array_like
        Objective function coefficients
    A : array_like
        Constraint matrix
    b : array_like
        Right-hand side of constraints
    max_iter : int, optional
        Maximum number of iterations
    tol : float, optional
        Tolerance for convergence
    mu : float, optional
        Path parameter (between 0 and 1)
    
    Returns:
    --------
    x : ndarray
        Optimal primal solution
    y : ndarray
        Optimal dual solution
    s : ndarray
        Optimal slack variables
    """
    
    # Convert inputs to numpy arrays
    c = np.array(c, dtype=float)
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    
    m, n = A.shape
    
    # Initialize variables
    x = np.ones(n)  # Primal variables
    y = np.zeros(m)  # Dual variables
    s = np.ones(n)  # Slack variables
    
    # Main loop
    for iteration in range(max_iter):
        # Current duality measure
        mu_current = np.dot(x, s) / n
        
        # Check convergence
        primal_residual = np.dot(A, x) - b
        dual_residual = np.dot(A.T, y) + s - c
        complementarity = x * s
        
        if (norm(primal_residual) < tol and 
            norm(dual_residual) < tol and 
            norm(complementarity) < tol):
            break
            
        # Compute search direction
        X = np.diag(x)
        S = np.diag(s)
        
        # Form the KKT matrix
        zero_m = np.zeros((m, m))
        row1 = np.hstack([S, np.zeros((n, m)), X])
        row2 = np.hstack([A, np.eye(m), np.zeros((m, n))])
        row3 = np.hstack([np.zeros((n, n)), A.T, np.eye(n)])
        KKT = np.vstack([row1, row2, row3])
        
        # Compute right-hand side
        sigma = mu  # Centering parameter
        rc = -complementarity + sigma * mu_current
        rp = -primal_residual
        rd = -dual_residual
        rhs = np.concatenate([rc, rp, rd])
        
        # Solve KKT system
        solution = solve(KKT, rhs)
        dx = solution[:n]
        dy = solution[n:n+m]
        ds = solution[n+m:]
        
        # Line search
        alpha_primal = 0.99995 * min(1, min(-x[dx < 0] / dx[dx < 0]) if any(dx < 0) else 1)
        alpha_dual = 0.99995 * min(1, min(-s[ds < 0] / ds[ds < 0]) if any(ds < 0) else 1)
        
        # Update variables
        x += alpha_primal * dx
        y += alpha_dual * dy
        s += alpha_dual * ds
    
    return x, y, s

# Example usage
if __name__ == "__main__":
    # Example problem:
    # minimize    -x1 - x2
    # subject to   x1 + x2 = 1
    #              x1, x2 >= 0
    
    c = np.array([-1, -1])
    A = np.array([[1, 1]])
    b = np.array([1])
   
    # Alternate problem
    A = np.array([[1, 2, 3], [4, 5, 6]])
    b = np.array([7, 8])
    c = np.array([1, 2, 3])
    
    x, y, s = primal_dual_ipm(c, A, b)
    
    print("Optimal solution:")
    print(f"x = {x}")
    print(f"y = {y}")
    print(f"s = {s}")
    print(f"Objective value = {np.dot(c, x)}")

Optimal solution:
x = [0.         0.         1.53333333]
y = [ 23998.25475387 -11998.62737693]
s = [23997.25475387 11998.62737693     0.        ]
Objective value = 4.600000000000727


In [7]:
import numpy as np
from numpy.linalg import norm, solve

def primal_dual_ipm(c: np.ndarray, A: np.ndarray, b: np.ndarray, 
                    max_iter: int = 10000, tol: float = 1e-8, mu: float = 0.1, verbose: bool = False):
    """
    Solve Linear Programming problem using Primal-Dual Path-Following Interior Point Method.
    
    minimize    c^T x
    subject to  Ax = b
                x >= 0
    """
    c, A, b = map(np.array, [c, A, b])
    m, n = A.shape
    
    # Validate dimensions
    assert c.shape == (n,), "c must have shape (n,)"
    assert b.shape == (m,), "b must have shape (m,)"
    
    # Initialize variables
    x = np.ones(n)
    y = np.zeros(m)
    s = np.ones(n)
    
    for iteration in range(max_iter):
        # Compute residuals
        residual_primal = np.dot(A, x) - b
        residual_dual = np.dot(A.T, y) + s - c
        complementarity = x * s
        duality_gap = np.dot(x, s) / n
        
        # Check for convergence
        if (norm(residual_primal) < tol and 
            norm(residual_dual) < tol and 
            duality_gap < tol):
            if verbose:
                print(f"Converged at iteration {iteration}.")
            break
        
        # Compute KKT matrix
        X = np.diag(x)
        S = np.diag(s)
        KKT_matrix = np.block([
            [S, np.zeros((n, m)), X],
            [A, np.eye(m), np.zeros((m, n))],
            [np.zeros((n, n)), A.T, np.eye(n)],
        ])
        
        # Compute RHS
        sigma = mu
        rc = -complementarity + sigma * duality_gap
        rhs = np.concatenate([-rc, -residual_primal, -residual_dual])
        
        # Solve the system
        delta = solve(KKT_matrix, rhs)
        delta_x = delta[:n]
        delta_y = delta[n:n + m]
        delta_s = delta[n + m:]
        
        # Line search
        alpha_primal = 0.99995 * min(1, np.min(-x[delta_x < 0] / delta_x[delta_x < 0], initial=1))
        alpha_dual = 0.99995 * min(1, np.min(-s[delta_s < 0] / delta_s[delta_s < 0], initial=1))
        
        # Update variables
        x += alpha_primal * delta_x
        y += alpha_dual * delta_y
        s += alpha_dual * delta_s
        
        if verbose:
            print(f"Iter {iteration}: primal_residual={norm(residual_primal):.2e}, "
                  f"dual_residual={norm(residual_dual):.2e}, duality_gap={duality_gap:.2e}")
    
    return x, y, s

# Example usage
if __name__ == "__main__":
    # Alternate problem
    A = np.array([[1, 2, 3], [4, 5, 6]])
    b = np.array([7, 8])
    c = np.array([1, 2, 3])
    
    x, y, s = primal_dual_ipm(c, A, b)
    
    print("Optimal solution:")
    print(f"x = {x}")
    print(f"y = {y}")
    print(f"s = {s}")
    print(f"Objective value = {np.dot(c, x)}")


Optimal solution:
x = [0.         0.53949095 1.07930419]
y = [-12729.11716441 -94048.31756335]
s = [388923.38741781 495701.82214556 602480.25687332]
Objective value = 4.31689447998032


In [4]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Create the problem
problem = LpProblem("Simple_MIP_Problem", LpMaximize)

# Define decision variables
x = LpVariable("x", lowBound=0, cat="Continuous")  # x is an integer variable
y = LpVariable("y", lowBound=0, cat="Continuous")  # y is a continuous variable
z = LpVariable("z", lowBound=0, cat="Continuous")  # z is a continuous variable


# Define the objective function
problem += 1 * x + 2 * y + 3 * z, "Objective"

# Define the constraints
problem +=  1 * x + 2 * y + 3 * z <= 7, "Constraint_1"
problem +=  4 * x + 5 * y + 6 * z <= 8, "Constraint_2"

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)
print("Optimal Solution:")
print(f"x = {x.varValue}")
print(f"y = {y.varValue}")
print(f"z = {z.varValue}")

print("Objective value =", problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/vivekchaudhary/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/eb290dbd799046a2846ab4330b0fd74a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/eb290dbd799046a2846ab4330b0fd74a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 17 RHS
At line 20 BOUNDS
At line 21 ENDATA
Problem MODEL has 2 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 3 (0) columns and 6 (0) elements
0  Obj -0 Dual inf 6.8999997 (3)
0  Obj -0 Dual inf 6.8999997 (3)
1  Obj 4
Optimal - objective value 4
Optimal objective 4 - 1 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock